<a href="https://colab.research.google.com/github/Junjaee/Study/blob/main/SQL/Python_SQL_%EC%97%B0%EB%8F%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 1. 파이썬과 MySQL 활용

- 파이썬과 MySQL을 연결해주는 모듈 : mysqlclient, PyMySQL
  - pip install mysqlclient
  - pip install PyMySQL



In [ ]:
pip install mysqlclient

In [ ]:
pip install PyMySQL

     |████████████████████████████████| 51kB 2.6MB/s 


In [ ]:
# mysqlclient 모듈 사용

import MySQLdb

conn = MySQLdb.connect(host = "127.0.0.1", user = "root", password = "1111", database = "world")
print(conn)
conn.close()

In [ ]:
# pymysql 모듈 사용

import pymysql

conn = pymysql.connect(host = "127.0.0.1", user = "root", password = "1111", database = "world")
print(conn)
conn.close()

In [ ]:
# mysqlclient 모듈을 사용해서 계속 실습하기

config = {"host" : "127.0.0.1", "user" : "root", "password" : "1111", "database" : "world"}

### (1) SELECT

In [ ]:
conn = MySQLdb.connect(**config)

sql = 'select * from city limit 10'

cur = conn.cursor()
cur.execute(sql)

# print(cur)
# for data in cur:
#     print(data)
#     print("%d %s %s %s %d"% data)

# for a,b,c,d,e in cur:
#     print(a,b,c,d,e)


result = list()
for data in cur:
    result.append(data)
       
# result = list(data for data in cur)    
    
conn.close()

print(result)

### (2) INSERT

In [ ]:
conn = MySQLdb.connect(**config)

sql = "create table if not exists tbltest(empno int, ename varchar(10), sal int, job varchar(10))"
# if not exists -> 테이블 중복 생성시 오류 방지

cur = conn.cursor()
cur.execute(sql)

sql = "insert into tbltest(empno, ename, sal) values(1, 'scott', 5000)"
cur.execute(sql)
conn.commit()

conn.close()

In [ ]:
empno = 2
ename = 'miller'
sal = 1300

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values(" + str(empno) + ", '" + ename + "', " + str(sal) + ")"
# miller 에 따옴표를 해주어야 문자로 인식한다
print(sql)
cur.execute(sql)
conn.commit()

conn.close()

In [ ]:
empno = 3
ename = 'tom'
sal = 2000

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
sql_data = (str(empno), ename, str(sal))

cur.execute(sql, sql_data)
conn.commit()

conn.close()
# close를 안하면 서버에 부하가 가해진다

In [ ]:
try : 
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    empno = 4
    ename = 'jerry'
    sal = 2000
    
    sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
    sql_data = (str(empno), ename, str(sal))

    cur.execute(sql, sql_data)
    conn.commit()
except Exception as e:
    print("error : ", e)
finally:
    conn.close()

In [ ]:
# 데이터 여러 개 입력하기

conn = MySQLdb.connect(**config)
cur = conn.cursor()

for i in range(10):
    sql = "insert into tbltest(empno, ename, sal) values(5, 'jane', 10000)"
    cur.execute(sql)
    conn.commit()
    
conn.close()

### (3) cursor 객체의 fetchall(), fetchmany(), fetchone()
- default는 fetchall()

In [ ]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("select * from tbltest")
    
#     for i in cur.fetchall(10):
#         print(i)
        
#     for i in cur.fetchmany(5):
#         print(i)
        
    print(cur.fetchone())
    # fetchmany(1)보다 성능적으로 더 좋다
    
    
finally:
    conn.close()

### (4) DictCursor

- 필드명을 인덱스 대신 키로 사용

In [ ]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor(MySQLdb.cursors.DictCursor)
    
    cur.execute("select * from tbltest")
    
    for i in cur.fetchall():
        print(i["empno"], i["ename"], i["sal"])

        
finally:
    conn.close()

### (5) 응용 실습1 : CSV파일을 DB에 저장

In [ ]:
'''
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
'''

In [ ]:
import csv

file = csv.reader(open("D:\Coding\Acorn\SQL\응용 실습용 CSV파일.csv"))
print(file)

# print(next(file))

header = next(file)

try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("delete from suppliers")
    conn.commit()
    #이미 테이블에 등록되어 있는 데이터들 전부 삭제하기
    
    for row in file:
#         print(row)
        cur.execute("insert into suppliers values(%s, %s, %s, %s, %s)", row)
    conn.commit()
        
finally:
    conn.close()

### (6) DB에서 가져온 데이터를 파일로 저장

In [ ]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("select * from suppliers where cost > 500000")
    
#     for i in cur:
#         print(i)
    file = csv.writer(open("D:\Coding\Acorn\SQL\응용 실습용 CSV파일1.csv", "w"), delimiter = ",")
    # delimeter -> 데이터 사이사이를 무엇으로 나눌것인가
    
    header = ["Supplier Name", "Invoice Number", "Part Number", "Cost", "Purchase Date"]
    file.writerow(header)
    
    for row in cur:
        file.writerow(row)
        
    
    
finally:
    conn.close()

In [ ]:
D:\Coding\Acorn\SQL\dbProject\mysql2.py 참고

wxFormBuilder를 이용한 DB 프로그램 만들기 실습

In [ ]:
# 실습을 위한 데이터 베이스 생성

create database pywin;
use pywin;

ALTER DATABASE pywin DEFAULT CHARACTER SET utf8;
-- ALTER TABLE 테이블명 CONVERT TO CHARACTER SET utpywinf8;

create table product(code int primary key,
            product_name varchar(20),
            product_count int,
            product_price int);

insert into product values(1,'장갑',3,5000);
insert into product values(2,'가죽장갑',10,50000);
insert into product values(3,'가죽점퍼',5,650000);

create table Departure(
          dep_no int(4)  primary key,
          dep_name varchar(10) not null,
          dep_loc varchar(10),
          dep_tel varchar(15));

insert into Departure values(10,'총무부','서울','02-100-1111');
insert into Departure values(20,'영업부','서울','02-100-2222');
insert into Departure values(30,'전산부','서울','02-100-3333');

create table emp(emp_no int(4) primary key,
          emp_name varchar(10) not null,
          dep_num int(4) not null,
          emp_jik varchar(10) default '사원',
          emp_pay int,
          emp_hiredate date,
          emp_gen char(3),
          CONSTRAINT ck_sawon_gen check(emp_gen='남' or emp_gen='여'));

insert into emp values(1,'홍길동',10,'사장',8000,'98/09/01','남');
insert into emp values(2,'한국남',20,'부장',6200,'01/01/03','남');
insert into emp values(3,'이순신',20,'과장',4500,'01/03/03','남');
insert into emp values(4,'이미라',30,'대리',3503,'04/01/04','여');
insert into emp values(5,'이순라',20,'사원',2850,'09/08/05','여');

create table customer(cus_no int(4) primary key,
          cus_name varchar(10) not null,
          cus_tel varchar(20),
          cus_jumin char(14),
          cus_damsano int(4),
          CONSTRAINT FK_gogek_damsano foreign key(cus_damsano)
          references emp(emp_no));

insert into customer values(1,'이나라','02-535-2580','850612-1156789',1);
insert into customer values(2,'김혜순','02-375-6946','700101-1054225',3);
insert into customer values(3,'최부자','02-692-8926','890305-1065773',3);
insert into customer values(4,'김해자','032-393-6277','770412-2028677',1);
insert into customer values(5,'차일호','02-294-2946','790509-1062677',2);

In [ ]:
# -*- coding: utf-8 -*- 

###########################################################################
## Python code generated with wxFormBuilder (version Jun 17 2015)
## http://www.wxformbuilder.org/
##
## PLEASE DO "NOT" EDIT THIS FILE!
###########################################################################

import wx
import wx.xrc
import MySQLdb

config = {"host" : "127.0.0.1", "user" : "root", "password" : "1111", "database" : "pywin", "port" : 3306, "charset" : "utf8", "use_unicode" : True}


###########################################################################
## Class MainFrame
###########################################################################

class MainFrame ( wx.Frame ):
	
	def __init__( self, parent ):
		wx.Frame.__init__ ( self, parent, id = wx.ID_ANY, title = wx.EmptyString, pos = wx.DefaultPosition, size = wx.Size( 374,456 ), style = wx.DEFAULT_FRAME_STYLE|wx.TAB_TRAVERSAL )
		
		self.SetSizeHintsSz( wx.DefaultSize, wx.DefaultSize )
		
		bSizer2 = wx.BoxSizer( wx.VERTICAL )
		
		self.m_panel2 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer3 = wx.BoxSizer( wx.HORIZONTAL )
		
		self.m_staticText2 = wx.StaticText( self.m_panel2, wx.ID_ANY, u"번호 : ", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staticText2.Wrap( -1 )
		bSizer3.Add( self.m_staticText2, 0, wx.ALL, 5 )
		
		self.m_txtNo = wx.TextCtrl( self.m_panel2, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer3.Add( self.m_txtNo, 0, wx.ALL, 5 )
		
		self.m_btnInsert = wx.Button( self.m_panel2, wx.ID_ANY, u"등록", wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer3.Add( self.m_btnInsert, 1, wx.ALL, 5 )
		
		
		self.m_panel2.SetSizer( bSizer3 )
		self.m_panel2.Layout()
		bSizer3.Fit( self.m_panel2 )
		bSizer2.Add( self.m_panel2, 0, wx.EXPAND |wx.ALL, 5 )
		
		self.m_panel3 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer4 = wx.BoxSizer( wx.HORIZONTAL )
		
		self.m_staticText3 = wx.StaticText( self.m_panel3, wx.ID_ANY, u"이름 : ", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staticText3.Wrap( -1 )
		bSizer4.Add( self.m_staticText3, 0, wx.ALL, 5 )
		
		self.m_txtName = wx.TextCtrl( self.m_panel3, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer4.Add( self.m_txtName, 0, wx.ALL, 5 )
		
		self.m_btnUpdate = wx.Button( self.m_panel3, wx.ID_ANY, u"수정", wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer4.Add( self.m_btnUpdate, 0, wx.ALL, 5 )
		
		self.m_btnConfirm = wx.Button( self.m_panel3, wx.ID_ANY, u"확인", wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer4.Add( self.m_btnConfirm, 0, wx.ALL, 5 )
		
		
		self.m_panel3.SetSizer( bSizer4 )
		self.m_panel3.Layout()
		bSizer4.Fit( self.m_panel3 )
		bSizer2.Add( self.m_panel3, 0, wx.EXPAND |wx.ALL, 5 )
		
		self.m_panel4 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer5 = wx.BoxSizer( wx.HORIZONTAL )
		
		self.m_staticText4 = wx.StaticText( self.m_panel4, wx.ID_ANY, u"전화 : ", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staticText4.Wrap( -1 )
		bSizer5.Add( self.m_staticText4, 0, wx.ALL, 5 )
		
		self.m_txtTel = wx.TextCtrl( self.m_panel4, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer5.Add( self.m_txtTel, 0, wx.ALL, 5 )
		
		self.m_btnDel = wx.Button( self.m_panel4, wx.ID_ANY, u"삭제", wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer5.Add( self.m_btnDel, 1, wx.ALL, 5 )
		
		
		self.m_panel4.SetSizer( bSizer5 )
		self.m_panel4.Layout()
		bSizer5.Fit( self.m_panel4 )
		bSizer2.Add( self.m_panel4, 0, wx.EXPAND |wx.ALL, 5 )
		
		self.m_panel5 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer6 = wx.BoxSizer( wx.VERTICAL )
		
		self.m_lstView = wx.ListCtrl( self.m_panel5, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.LC_REPORT )
		bSizer6.Add( self.m_lstView, 1, wx.ALL|wx.EXPAND, 5 )
		
        # 리스트 컨트롤 화면 디자인
		self.m_lstView.InsertColumn(0, "번호", width = 50)
		self.m_lstView.InsertColumn(1, "이름", width = 100)
		self.m_lstView.InsertColumn(2, "전화", width = 100)

		
		self.m_panel5.SetSizer( bSizer6 )
		self.m_panel5.Layout()
		bSizer6.Fit( self.m_panel5 )
		bSizer2.Add( self.m_panel5, 1, wx.EXPAND |wx.ALL, 5 )
		
		self.m_panel6 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer7 = wx.BoxSizer( wx.HORIZONTAL )
		
		self.m_staticText5 = wx.StaticText( self.m_panel6, wx.ID_ANY, u"인원수 : ", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staticText5.Wrap( -1 )
		bSizer7.Add( self.m_staticText5, 0, wx.ALL, 5 )
		
		self.m_staCnt = wx.StaticText( self.m_panel6, wx.ID_ANY, u"0", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staCnt.Wrap( -1 )
		bSizer7.Add( self.m_staCnt, 0, wx.ALL, 5 )
		
		
		self.m_panel6.SetSizer( bSizer7 )
		self.m_panel6.Layout()
		bSizer7.Fit( self.m_panel6 )
		bSizer2.Add( self.m_panel6, 0, wx.EXPAND |wx.ALL, 5 )
		
		
		self.SetSizer( bSizer2 )
		self.Layout()

		self.Centre(wx.BOTH)

		# 버튼 이벤트 id를 지정
		self.m_btnInsert.id = 1
		self.m_btnUpdate.id = 2
		self.m_btnConfirm.id = 3
		self.m_btnDel.id = 4

		# 이벤트 바인딩
		self.m_btnInsert.Bind(wx.EVT_BUTTON, self.onBtnClick)
		self.m_btnUpdate.Bind(wx.EVT_BUTTON, self.onBtnClick)
		self.m_btnConfirm.Bind(wx.EVT_BUTTON, self.onBtnClick)
		self.m_btnDel.Bind(wx.EVT_BUTTON, self.onBtnClick)

		self.m_btnConfirm.Enable(enable = False)
		self.viewListData()

	def onBtnClick(self, e):
		id = e.GetEventObject().id

		if id == 1:  # 등록 버튼
			self.cusInsert()
		elif id == 2:  # 수정 버튼
			self.cusUpdate()
		elif id == 3:  # 확인 버튼
			self.cusUpdateOK()
		elif id == 4:  # 삭제 버튼
			self.cusDel()
		elif id == 5:  # 취소 버튼
			self.cusUpdateCancel()

    
    # ListCtrl에 고객 정보 출력
	def viewListData(self):
		try:
			conn = MySQLdb.connect(**config)
			cur = conn.cursor()

			self.m_lstView.DeleteAllItems()

			cur.execute("select * from customer")
            
			count = 0            
			for row in cur:
				i = self.m_lstView.InsertItem(10000, 0)
				self.m_lstView.SetItem(i, 0, str(row[0]))
				self.m_lstView.SetItem(i, 1, str(row[1]))
				self.m_lstView.SetItem(i, 2, str(row[2]))
				count += 1

			self.m_staCnt.SetLabel(str(count))

		except Exception as e:
			print("viewListData에서 오류 : ", e)
		finally:
			conn.close()

    # 중복된 고객번호 체크
	def selectData(self, no):
		try:
			conn = MySQLdb.connect(**config)
			cur = conn.cursor()

			sql = "select cus_no, cus_name, cus_tel from customer where cus_no = {}".format(no)
			cur.execute(sql)

			data = cur.fetchone()
			return data

		except Exception as e:
			print("viewListData에서 오류 : ", e)
		finally:
			conn.close()        

	
    # 고객 등록
	def cusInsert(self):
		no = self.m_txtNo.GetValue()
		name = self.m_txtName.GetValue()
		tel = self.m_txtTel.GetValue()

		if no == "" or name == "" or tel == "":
			wx.MessageBox("자료를 입력하세요", "입력", wx.OK)
			return
 
		try:
			conn = MySQLdb.connect(**config)
			cur = conn.cursor()

			result = self.selectData(no)
			if result != None:
				wx.MessageBox("이미 사용중인 번호입니다", "알림", wx.OK)
				return

			sql = "insert into customer(cus_no, cus_name, cus_tel) values(%s, %s, %s)"
			cur.execute(sql, (no, name, tel))
			conn.commit()

			self.viewListData()
			self.m_txtNo.SetValue("")
			self.m_txtName.SetValue("")
			self.m_txtTel.SetValue("")

		except Exception as e:
			print("viewListData에서 오류 : ", e)
		finally:
			conn.close()

	# 고객 수정
    
	def cusUpdate(self):
		dlg = wx.TextEntryDialog(self, "수정할 번호 입력", "수정")
		if dlg.ShowModal() == wx.ID_OK:
			if dlg.GetValue() == "":
				return

			up_no = dlg.GetValue()

			result = self.selectData(up_no)
			if result == None:
				wx.MessageBox(up_no + "번은 등록된 자료가 아닙니다.", "알림", wx.OK)
				return

			self.m_txtNo.SetValue(str(result[0]))
			self.m_txtName.SetValue(str(result[1]))
			self.m_txtTel.SetValue(str(result[2]))

			self.m_txtNo.SetEditable(False)
			self.m_btnConfirm.Enable(True)
			self.m_btnUpdate.SetLabel("취소")
			self.m_btnUpdate.id = 5


	# 고객 수정 완료
	def cusUpdateOK(self):
		no = self.m_txtNo.GetValue()
		name = self.m_txtName.GetValue()
		tel = self.m_txtTel.GetValue()

		if name == "" or tel == "":
			wx.MessageBox("자료를 입력하세요", "입력", wx.OK)
			return
 
		try:
			conn = MySQLdb.connect(**config)
			cur = conn.cursor()

			sql = "update customer set cus_name = %s, cus_tel = %s where cus_no = %s"
			cur.execute(sql, (name, tel, no))
			conn.commit()

			self.viewListData()
			self.m_txtNo.SetValue("")
			self.m_txtName.SetValue("")
			self.m_txtTel.SetValue("")

			self.m_txtNo.SetEditable(True)
			self.m_btnUpdate.SetLabel("수정")
			self.m_btnUpdate.id = 2
			self.m_btnConfirm.Enable(False)

		except Exception as e:
			print("cusUpdateOK에서 오류 : ", e)
		finally:
			conn.close()


	# 고객 수정 취소
	def cusUpdateCancel(self):
		self.m_txtNo.SetValue("")
		self.m_txtName.SetValue("")
		self.m_txtTel.SetValue("")

		self.m_txtNo.SetEditable(True)
		self.m_btnUpdate.SetLabel("수정")
		self.m_btnUpdate.id = 2
		self.m_btnConfirm.Enable(False)

	# 고객 삭제

	def cusDel(self):
		dlg = wx.TextEntryDialog(self, "삭제할 번호 입력", "삭제")
		if dlg.ShowModal() == wx.ID_OK:
			if dlg.GetValue() == "":
				return

			del_no = dlg.GetValue()

			result = self.selectData(del_no)
			if result == None:
				wx.MessageBox(del_no + "번은 등록된 자료가 아닙니다.", "알림", wx.OK)
				return

			try:
				conn = MySQLdb.connect(**config)
				cur = conn.cursor()

				sql = "delete from customer where cus_no = {}".format(del_no)
				cur.execute(sql)
				conn.commit()
				self.viewListData()

			except Exception as e:
				print("cusDel에서 오류 : ", e)
			finally:
				conn.close()


if __name__ == "__main__":
	app = wx.App()
	MainFrame(None).Show(True)
	app.MainLoop()


## 2. 파이썬과 SQLite

- 1) 로컬로 사용할 수 있는 DBMS
    - Access
    - SQLite
    - Microsoft SQL Server Compact Edition
    - Firebird Embedded Server
    - H2
    - HSQL
    - Apache Derby
    - ....

- 2) SQLite 특징
    - 파일 기반의 DBMS, 저메모리, 빠른 처리속도
    - 오픈 소스
    - 별도의 DB서버가 없어도 쉽고 편리하게 사용할 수 있는 Embeded SQL DB엔진
    - 안드로이드, 아이폰 등의 스마트폰에 기본 내장된 DB
    - 파이썬에도 내장
    - 표준SQL 지원
    -------------------------------
    - 컬럼을 삭제하거나 변경하는 것 등이 제한
    - Left Outer Join만 지원
    - 읽기 전용 뷰만 제공
    - ...

- 3) 공식 사이트
    - http://sqlite.org

- 4) 클라이언트 툴
    - http://www.sqliteexpert.com

In [ ]:
# sqlite3를 이용해 db를 추가해 보기

CREATE TABLE tblBook(
                            author  varchar(20),
                            title varchar(20)
    );
 
    INSERT INTO tblBook VALUES('최주현', '하늘과 땅');
    INSERT INTO tblBook VALUES('최주현', '바다');
    INSERT INTO tblBook VALUES('유은정', '바다');
    INSERT INTO tblBook VALUES('박성우', '문');
    INSERT INTO tblBook VALUES('최주현', '문');
    INSERT INTO tblBook VALUES('박성우', '천국');
    INSERT INTO tblBook VALUES('최지은', '천국');
    INSERT INTO tblBook VALUES('최주현', '천국');
    INSERT INTO tblBook VALUES('박성우', '고슴도치');
    INSERT INTO tblBook VALUES('서금동', '나');

In [ ]:
import sqlite3

In [ ]:
print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.22.0


In [ ]:
conn = sqlite3.connect("D:\Coding\Acorn\SQL\SQLitetest\sqlitetest.db")
print(conn)
conn.close()

In [ ]:
# select

conn = sqlite3.connect("D:\Coding\Acorn\SQL\SQLitetest\sqlitetest.db")
cur = conn.cursor()

cur.execute("select * from tblbook")
for row in cur:
    print(row)

conn.close()

In [ ]:
# 자동으로 db 생성하고 테이블 생성

conn = sqlite3.connect("D:\Coding\Acorn\SQL\SQLitetest\\auto.db")
print(conn)
cur = conn.cursor()

cur.execute("create table if not exists tblmember(id    int, name  varchar(10))")
conn.commit()

cur.execute("insert into tblmember values(1, '홍길동')")
conn.commit()

cur.execute("select * from tblMember")
for row in cur:
    print(row)

conn.close()

In [ ]:
####### csv 파일을 읽어들여서 DB에 저장 #######
import sqlite3
import csv

conn = sqlite3.connect("D:\Coding\Acorn\SQL\SQLitetest\sqlitetest.db")
cur = conn.cursor()

# 테이블 조회
tables = cur.execute("select * from sqlite_master where type = 'table'")
for i in tables:
    print(i)
    
# 파일 불러오기
file = csv.reader(open("D:\Coding\Acorn\SQL\응용 실습용 CSV파일.csv", "r"), delimiter = ",")
# for i in file:
#     print(i)

next(file)

for row in file:
    cur.execute("insert into suppliers values(?, ?, ?, ?, ?)", row)
    
conn.commit()

conn.close()

In [ ]:
# expert로 확인하거나 select로 확인
conn = sqlite3.connect("D:\Coding\Acorn\SQL\SQLitetest\sqlitetest.db")
cur = conn.cursor()

cur.execute("select * from suppliers")
for i in cur.fetchmany(5):
    print(i)
    
conn.close()

In [ ]:
###### Memory DB ######

conn = sqlite3.connect(":memory:")
# print(conn)
cur = conn.cursor()

cur.execute("create table phonebook(name varchar(10), phonenumber text);")
cur.execute("insert into phonebook values(?, ?);", ("tom", "010-111-1111"))

# 입력 데이터 인자를 변수명으로 처리
sql = "insert into phonebook values(:vName, :vPhone)"
cur.execute(sql, {"vPhone":"010-222-2222", "vName":"Johnson"})

# 여러 개의 데이터를 한번에 입력
datalist = (('jane', '010-3333-3333'), ('jerry', '010-4444-4444'), ('marry', '010-5555-5555'))
cur.executemany("insert into phonebook values(?, ?)", datalist)


cur.execute("select * from phonebook")
for row in cur:
    print(row)
    
# 지금 작업 내용을 script로 저장
f = open("D:\Coding\Acorn\SQL\SQLitetest\dump.db", "w")
for line in conn.iterdump():
    f.write("{}\n".format(line))
    
f.close()

conn.close()

In [ ]:
# 스크립트 불러오기
conn = sqlite3.connect("D:\Coding\Acorn\SQL\SQLitetest\sqlitetest.db")
cur = conn.cursor()

file = open("D:\Coding\Acorn\SQL\SQLitetest\dump.db", "r")
script = file.read()
print(script)

cur.executescript(script)

conn.close()

In [ ]:
#vsCode에서 실행해보기

import sqlite3

def process(dbname):
    try :
        conn = sqlite3.connect(dbname)
        cur = conn.cursor()

        sql = "drop table if exists emp"
        cur.execute(sql)

        sql = "create table emp(id integer primary key, name text)"
        cur.execute(sql)

        # tables = cur.execute("select * from sqlite_master where type = 'table'")
        # for i in tables:
        #     print(i)

        cur.execute("insert into emp values(1, '홍길동')")
        cur.execute("insert into emp values(?, ?)", (2, "임꺽정"))
        cur.execute("insert into emp values(?, ?)", [3, "신돌석"])

        tlist = ((4, "유비"), (5, "관우"), (6, "장비"))
        cur.executemany("insert into emp values(?, ?)", tlist)

        cur.execute("insert into emp values(:no, :name)", {"no" : 7, "name" : "호랑이"})

        cur.execute("select * from emp")
        for row in cur:
            print(row)

        print("===========================")

        cur.execute("select count(*) from emp")
        print("데이터 갯수 : " + str(cur.fetchone()))

        conn.commit()

    except sqlite3.Error as e:
        print("오류 : ", e)
        conn.rollback()
    finally : 
        conn.close()

if __name__ == "__main__":
    process("nice.db")

In [ ]:
#MySQLdb에서 했던 실습을 sqlite3에서 해보기
# MySQLdb --> sqlite3
# config --> dbname
# %s --> ? 로 전부 바꿔주기


# -*- coding: utf-8 -*- 

###########################################################################
## Python code generated with wxFormBuilder (version Jun 17 2015)
## http://www.wxformbuilder.org/
##
## PLEASE DO "NOT" EDIT THIS FILE!
###########################################################################

import wx
import wx.xrc
import sqlite3

dbname = "D:\Coding\Acorn\SQL\SQLitetest\pywin.db"


###########################################################################
## Class MainFrame
###########################################################################

class MainFrame ( wx.Frame ):
	
	def __init__( self, parent ):
		wx.Frame.__init__ ( self, parent, id = wx.ID_ANY, title = wx.EmptyString, pos = wx.DefaultPosition, size = wx.Size( 374,456 ), style = wx.DEFAULT_FRAME_STYLE|wx.TAB_TRAVERSAL )
		
		self.SetSizeHintsSz( wx.DefaultSize, wx.DefaultSize )
		
		bSizer2 = wx.BoxSizer( wx.VERTICAL )
		
		self.m_panel2 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer3 = wx.BoxSizer( wx.HORIZONTAL )
		
		self.m_staticText2 = wx.StaticText( self.m_panel2, wx.ID_ANY, u"번호 : ", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staticText2.Wrap( -1 )
		bSizer3.Add( self.m_staticText2, 0, wx.ALL, 5 )
		
		self.m_txtNo = wx.TextCtrl( self.m_panel2, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer3.Add( self.m_txtNo, 0, wx.ALL, 5 )
		
		self.m_btnInsert = wx.Button( self.m_panel2, wx.ID_ANY, u"등록", wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer3.Add( self.m_btnInsert, 1, wx.ALL, 5 )
		
		
		self.m_panel2.SetSizer( bSizer3 )
		self.m_panel2.Layout()
		bSizer3.Fit( self.m_panel2 )
		bSizer2.Add( self.m_panel2, 0, wx.EXPAND |wx.ALL, 5 )
		
		self.m_panel3 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer4 = wx.BoxSizer( wx.HORIZONTAL )
		
		self.m_staticText3 = wx.StaticText( self.m_panel3, wx.ID_ANY, u"이름 : ", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staticText3.Wrap( -1 )
		bSizer4.Add( self.m_staticText3, 0, wx.ALL, 5 )
		
		self.m_txtName = wx.TextCtrl( self.m_panel3, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer4.Add( self.m_txtName, 0, wx.ALL, 5 )
		
		self.m_btnUpdate = wx.Button( self.m_panel3, wx.ID_ANY, u"수정", wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer4.Add( self.m_btnUpdate, 0, wx.ALL, 5 )
		
		self.m_btnConfirm = wx.Button( self.m_panel3, wx.ID_ANY, u"확인", wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer4.Add( self.m_btnConfirm, 0, wx.ALL, 5 )
		
		
		self.m_panel3.SetSizer( bSizer4 )
		self.m_panel3.Layout()
		bSizer4.Fit( self.m_panel3 )
		bSizer2.Add( self.m_panel3, 0, wx.EXPAND |wx.ALL, 5 )
		
		self.m_panel4 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer5 = wx.BoxSizer( wx.HORIZONTAL )
		
		self.m_staticText4 = wx.StaticText( self.m_panel4, wx.ID_ANY, u"전화 : ", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staticText4.Wrap( -1 )
		bSizer5.Add( self.m_staticText4, 0, wx.ALL, 5 )
		
		self.m_txtTel = wx.TextCtrl( self.m_panel4, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer5.Add( self.m_txtTel, 0, wx.ALL, 5 )
		
		self.m_btnDel = wx.Button( self.m_panel4, wx.ID_ANY, u"삭제", wx.DefaultPosition, wx.DefaultSize, 0 )
		bSizer5.Add( self.m_btnDel, 1, wx.ALL, 5 )
		
		
		self.m_panel4.SetSizer( bSizer5 )
		self.m_panel4.Layout()
		bSizer5.Fit( self.m_panel4 )
		bSizer2.Add( self.m_panel4, 0, wx.EXPAND |wx.ALL, 5 )
		
		self.m_panel5 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer6 = wx.BoxSizer( wx.VERTICAL )
		
		self.m_lstView = wx.ListCtrl( self.m_panel5, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.LC_REPORT )
		bSizer6.Add( self.m_lstView, 1, wx.ALL|wx.EXPAND, 5 )
		
        # 리스트 컨트롤 화면 디자인
		self.m_lstView.InsertColumn(0, "번호", width = 50)
		self.m_lstView.InsertColumn(1, "이름", width = 100)
		self.m_lstView.InsertColumn(2, "전화", width = 100)

		
		self.m_panel5.SetSizer( bSizer6 )
		self.m_panel5.Layout()
		bSizer6.Fit( self.m_panel5 )
		bSizer2.Add( self.m_panel5, 1, wx.EXPAND |wx.ALL, 5 )
		
		self.m_panel6 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
		bSizer7 = wx.BoxSizer( wx.HORIZONTAL )
		
		self.m_staticText5 = wx.StaticText( self.m_panel6, wx.ID_ANY, u"인원수 : ", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staticText5.Wrap( -1 )
		bSizer7.Add( self.m_staticText5, 0, wx.ALL, 5 )
		
		self.m_staCnt = wx.StaticText( self.m_panel6, wx.ID_ANY, u"0", wx.DefaultPosition, wx.DefaultSize, 0 )
		self.m_staCnt.Wrap( -1 )
		bSizer7.Add( self.m_staCnt, 0, wx.ALL, 5 )
		
		
		self.m_panel6.SetSizer( bSizer7 )
		self.m_panel6.Layout()
		bSizer7.Fit( self.m_panel6 )
		bSizer2.Add( self.m_panel6, 0, wx.EXPAND |wx.ALL, 5 )
		
		
		self.SetSizer( bSizer2 )
		self.Layout()

		self.Centre(wx.BOTH)

		# 버튼 이벤트 id를 지정
		self.m_btnInsert.id = 1
		self.m_btnUpdate.id = 2
		self.m_btnConfirm.id = 3
		self.m_btnDel.id = 4

		# 이벤트 바인딩
		self.m_btnInsert.Bind(wx.EVT_BUTTON, self.onBtnClick)
		self.m_btnUpdate.Bind(wx.EVT_BUTTON, self.onBtnClick)
		self.m_btnConfirm.Bind(wx.EVT_BUTTON, self.onBtnClick)
		self.m_btnDel.Bind(wx.EVT_BUTTON, self.onBtnClick)

		self.m_btnConfirm.Enable(enable = False)
		self.viewListData()

	def onBtnClick(self, e):
		id = e.GetEventObject().id

		if id == 1:  # 등록 버튼
			self.cusInsert()
		elif id == 2:  # 수정 버튼
			self.cusUpdate()
		elif id == 3:  # 확인 버튼
			self.cusUpdateOK()
		elif id == 4:  # 삭제 버튼
			self.cusDel()
		elif id == 5:  # 취소 버튼
			self.cusUpdateCancel()

    
    # ListCtrl에 고객 정보 출력
	def viewListData(self):
		try:
			conn = sqlite3.connect(dbname)
			cur = conn.cursor()

			self.m_lstView.DeleteAllItems()

			cur.execute("select * from customer")
            
			count = 0            
			for row in cur:
				i = self.m_lstView.InsertItem(10000, 0)
				self.m_lstView.SetItem(i, 0, str(row[0]))
				self.m_lstView.SetItem(i, 1, str(row[1]))
				self.m_lstView.SetItem(i, 2, str(row[2]))
				count += 1

			self.m_staCnt.SetLabel(str(count))

		except Exception as e:
			print("viewListData에서 오류 : ", e)
		finally:
			conn.close()

    # 중복된 고객번호 체크
	def selectData(self, no):
		try:
			conn = sqlite3.connect(dbname)
			cur = conn.cursor()

			sql = "select cus_no, cus_name, cus_tel from customer where cus_no = {}".format(no)
			cur.execute(sql)

			data = cur.fetchone()
			return data

		except Exception as e:
			print("viewListData에서 오류 : ", e)
		finally:
			conn.close()        

	
    # 고객 등록
	def cusInsert(self):
		no = self.m_txtNo.GetValue()
		name = self.m_txtName.GetValue()
		tel = self.m_txtTel.GetValue()

		if no == "" or name == "" or tel == "":
			wx.MessageBox("자료를 입력하세요", "입력", wx.OK)
			return
 
		try:
			conn = sqlite3.connect(dbname)
			cur = conn.cursor()

			result = self.selectData(no)
			if result != None:
				wx.MessageBox("이미 사용중인 번호입니다", "알림", wx.OK)
				return

			sql = "insert into customer(cus_no, cus_name, cus_tel) values(?, ?, ?)"
			cur.execute(sql, (no, name, tel))
			conn.commit()

			self.viewListData()
			self.m_txtNo.SetValue("")
			self.m_txtName.SetValue("")
			self.m_txtTel.SetValue("")

		except Exception as e:
			print("viewListData에서 오류 : ", e)
		finally:
			conn.close()

	# 고객 수정
    
	def cusUpdate(self):
		dlg = wx.TextEntryDialog(self, "수정할 번호 입력", "수정")
		if dlg.ShowModal() == wx.ID_OK:
			if dlg.GetValue() == "":
				return

			up_no = dlg.GetValue()

			result = self.selectData(up_no)
			if result == None:
				wx.MessageBox(up_no + "번은 등록된 자료가 아닙니다.", "알림", wx.OK)
				return

			self.m_txtNo.SetValue(str(result[0]))
			self.m_txtName.SetValue(str(result[1]))
			self.m_txtTel.SetValue(str(result[2]))

			self.m_txtNo.SetEditable(False)
			self.m_btnConfirm.Enable(True)
			self.m_btnUpdate.SetLabel("취소")
			self.m_btnUpdate.id = 5


	# 고객 수정 완료
	def cusUpdateOK(self):
		no = self.m_txtNo.GetValue()
		name = self.m_txtName.GetValue()
		tel = self.m_txtTel.GetValue()

		if name == "" or tel == "":
			wx.MessageBox("자료를 입력하세요", "입력", wx.OK)
			return
 
		try:
			conn = sqlite3.connect(dbname)
			cur = conn.cursor()

			sql = "update customer set cus_name = ?, cus_tel = ? where cus_no = ?"
			cur.execute(sql, (name, tel, no))
			conn.commit()

			self.viewListData()
			self.m_txtNo.SetValue("")
			self.m_txtName.SetValue("")
			self.m_txtTel.SetValue("")

			self.m_txtNo.SetEditable(True)
			self.m_btnUpdate.SetLabel("수정")
			self.m_btnUpdate.id = 2
			self.m_btnConfirm.Enable(False)

		except Exception as e:
			print("cusUpdateOK에서 오류 : ", e)
		finally:
			conn.close()


	# 고객 수정 취소
	def cusUpdateCancel(self):
		self.m_txtNo.SetValue("")
		self.m_txtName.SetValue("")
		self.m_txtTel.SetValue("")

		self.m_txtNo.SetEditable(True)
		self.m_btnUpdate.SetLabel("수정")
		self.m_btnUpdate.id = 2
		self.m_btnConfirm.Enable(False)

	# 고객 삭제

	def cusDel(self):
		dlg = wx.TextEntryDialog(self, "삭제할 번호 입력", "삭제")
		if dlg.ShowModal() == wx.ID_OK:
			if dlg.GetValue() == "":
				return

			del_no = dlg.GetValue()

			result = self.selectData(del_no)
			if result == None:
				wx.MessageBox(del_no + "번은 등록된 자료가 아닙니다.", "알림", wx.OK)
				return

			try:
				conn = sqlite3.connect(dbname)
				cur = conn.cursor()

				sql = "delete from customer where cus_no = {}".format(del_no)
				cur.execute(sql)
				conn.commit()
				self.viewListData()

			except Exception as e:
				print("cusDel에서 오류 : ", e)
			finally:
				conn.close()


if __name__ == "__main__":
	app = wx.App()
	MainFrame(None).Show(True)
	app.MainLoop()

## 3. 파이썬과 MongoDB 연동
  - pip install pymongo

In [2]:
# import pymongo
from pymongo import MongoClient

### (1) 서버 연결

In [ ]:
# conn = pymongo.MongoClient("123.0.0.1", 27017)
conn = MongoClient("123.0.0.1", 27017)
print(conn)
conn.close()

### (2) DB 연결

In [ ]:
conn = MongoClient("123,0,0,1", 27017)

# dbname = conn.get_database("test")
# dbname = conn.test
dbname = conn.test2
print(dbname)

conn.close()

### (3) Collection 연결

In [ ]:
conn = MongoClient("123.0.0.1", 27017)
dbname = conn.test

collect = dbname.articles
# collect = dbname["articles"]
print(collect)

conn.close()

### (4) 데이터 입력(1개)

In [ ]:
conn = MongoClient("127.0.0.1", 27017)
dbname = conn.test
collect = dbname.people

data = {"author":"Mike", "text":"my first blog post!", "tag":["mongodb", "python"]}
collect.insert_one(data)

# 입력하면서 id 가져오기
# data = {"author":"Mike", "text":"my first blog post!", "tag":["mongodb", "python"]}
# post_id = collect.insert_one(data).inserted_id
# print(post_id)

print(collect.count_documents({}))

for i in collect.find():
    print(i)

conn.close()

### (5) 데이터 입력하기(여러 개)

In [ ]:
conn = MongoClient("127.0.0.1", 27017)
dbname = conn.test
collect = dbname.people

print(collect.count_documents({}))

collect.insert_many([{"number": i} for i in range(10)])

print(collect.count_documents({}))

conn.close()

In [ ]:
datas = []
for i in range(10):
    data = {"number" : i}
    datas.append(data)
    
print(datas)

In [ ]:
conn = MongoClient("127.0.0.1", 27017)
dbname = conn.test
collect = dbname.people

data = list()
data.append({"name":"aron", "age":20})
data.append({"name":"tom", "age":21})
data.append({"name":"jane", "age":22})
data.append({"name":"jerry", "age":23})

collect.insert_many(data)

conn.close()

### (6) 조회

In [ ]:
conn = MongoClient("127.0.0.1", 27017)
dbname = conn.test
collect = dbname.people

# 전체 조회
# for i in collect.find():
#     print(i)
    
# 나이가 20살인 document 조회
for i in collect.find({"age" : 20}):
    print(i)
    
print("==========================")
    
# 나이가 23살 이상인 document 조회
for i in collect.find({"age" : {"$gte" : 23}}):
    print(i)

print("==========================")

print(collect.find_one())
print(collect.find_one({"age" : {"$gte" : 23}}))

conn.close()

### (7) 수정

In [ ]:
conn = MongoClient("127.0.0.1", 27017)
dbname = conn.test
collect = dbname.people

print(collect.find_one({"author" : "Tom"}))

collect.update_one({"author" : "Tom"}, {"$set" : {"text" : "my second blog post!"}})

print(collect.find_one({"author" : "Tom"}))

conn.close()